In [1]:
import tensorflow as tf
import os
from iexfinance.refdata import get_symbols

iex_token = os.environ['IEX_TOKEN']
if iex_token is None:
    raise Exception("IEX_TOKEN should be set")

get_symbols(output_format='pandas')

,symbol,exchange,name,date,type,iexId,region,currency,isEnabled,figi,cik
0,A,NYS,Agilent Technologies Inc.,2020-09-04,cs,IEX_46574843354B2D52,US,USD,True,BBG000C2V3D6,1090872
1,AA,NYS,Alcoa Corp.,2020-09-04,cs,IEX_4238333734532D52,US,USD,True,BBG00B3T3HD3,1675149
2,AAAU,PSE,Perth Mint Physical Gold ETF,2020-09-04,et,IEX_474B433136332D52,US,USD,True,BBG00LPXX872,None
3,AACG,NAS,ATA Creativity Global Sponsored ADR,2020-09-04,ad,IEX_44595A4C53392D52,US,USD,True,BBG000V2S3P6,None
4,AACQU,NAS,Artius Acquisition Inc. Units Cons of 1 Shs A ...,2020-09-04,ut,IEX_474656574E422D52,US,USD,True,BBG00VQPPKH7,None
...,...,...,...,...,...,...,...,...,...,...,...
8982,ZUO,NYS,Zuora Inc. Class A,2020-09-04,cs,IEX_4D474631525A2D52,US,USD,True,BBG000BT3HG5,1423774
8983,ZVO,NAS,Zovio Inc.,2020-09-04,cs,IEX_514A435352362D52,US,USD,True,BBG000C3CQP1,1305323
8984,ZYME,NYS,Zymeworks Inc.,2020-09-04,cs,IEX_5253315838322D52,US,USD,True,BBG00GD15LJ5,1403752
8985,ZYNE,NAS,Zynerba Pharmaceuticals Inc,2020-09-04,cs,IEX_4E3154424A382D52,US,USD,True,BBG007BBS8B7,1621443


In [2]:
from iexfinance.stocks import Stock

aapl = Stock("AAPL")
aapl.get_balance_sheet()

IEXQueryError: An error occurred while making the query.

In [3]:
from iexfinance.altdata import get_social_sentiment

get_social_sentiment("AAPL")

/Users/alexanderlerma/go/src/github.com/lermsai/notebooks/iex-cloud/venv/lib/python3.7/site-packages/iexfinance/altdata/__init__.py:48: UserWarning: UNSTABLE ENDPOINT: Not yet fully implemented by the provider.
  warnings.warn("UNSTABLE ENDPOINT: Not yet fully implemented by the "


IEXQueryError: An error occurred while making the query.

In [4]:
batch = Stock(["TSLA", "AAPL"])
batch.get_price()

IEXQueryError: An error occurred while making the query.

In [10]:
from datetime import datetime
from iexfinance.stocks import get_historical_data

start = datetime(2020, 1, 1)
end = datetime(2020, 9, 4)

df = get_historical_data("TSLA", start, end)

In [13]:
df.head()

,open,high,low,close,volume
date,,,,,
2020-01-02,84.90,86.14,84.34,86.05,47791930
2020-01-03,88.10,90.80,87.38,88.60,88973485
2020-01-06,88.09,90.31,88.00,90.31,50787495
2020-01-07,92.28,94.33,90.67,93.81,91045690
2020-01-08,94.74,99.70,93.65,98.43,155996965


In [14]:
df.tail()

,open,high,low,close,volume
date,,,,,
2020-08-28,459.02,463.70,437.30,442.68,100405880
2020-08-31,444.61,500.14,440.11,498.32,118374406
2020-09-01,502.14,502.49,470.51,475.05,90119419
2020-09-02,478.99,479.04,405.12,447.37,96176128
2020-09-03,407.23,431.80,402.00,407.00,87596086


In [16]:
df.plot()

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.